In [1]:
pip install pyspark

     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 198 kB 49.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=6d934dcdebce21b061f9d6d61491b8b7b01de21a482e3d3b0624f7643aee0cb6
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
import pyspark
spark = pyspark.sql.SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [4]:
spark

In [88]:
from pyspark.sql import Row
from pyspark.sql.types import *
crime_schema= StructType([
StructField('X',FloatType(), True),
StructField('Y', FloatType(),True),
StructField('RowID', IntegerType(), True),
StructField('CrimeDateTime',StringType(), True),
StructField('CrimeCode'	,StringType(), True),
StructField('Location'	,StringType(),True),
StructField('Description'	,StringType(), True),
StructField('Inside_Outside',StringType() , True),
StructField('Weapon',StringType(), True),
StructField('Post',IntegerType(), True),
StructField('District',StringType(), True),
StructField('Neighborhood',StringType(), True),
StructField('Latitude',FloatType(),True),
StructField('Longitude',FloatType(),True),
StructField('GeoLocation',StringType(), True),
StructField('Premise',StringType(),True),
StructField('VRIName',StringType(),True),
StructField('Total_Incidents',IntegerType(),True),
StructField('Shape',StringType(),True)])

In [89]:
crime_df = spark.read.csv("Part1_Crime_data.csv",  header=True, schema=crime_schema)

In [90]:
crime_df.cache()

DataFrame[X: float, Y: float, RowID: int, CrimeDateTime: string, CrimeCode: string, Location: string, Description: string, Inside_Outside: string, Weapon: string, Post: int, District: string, Neighborhood: string, Latitude: float, Longitude: float, GeoLocation: string, Premise: string, VRIName: string, Total_Incidents: int, Shape: string]

In [91]:
crime_df.count()

513158

In [92]:
crime_df.printSchema()

root
 |-- X: float (nullable = true)
 |-- Y: float (nullable = true)
 |-- RowID: integer (nullable = true)
 |-- CrimeDateTime: string (nullable = true)
 |-- CrimeCode: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Inside_Outside: string (nullable = true)
 |-- Weapon: string (nullable = true)
 |-- Post: integer (nullable = true)
 |-- District: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Latitude: float (nullable = true)
 |-- Longitude: float (nullable = true)
 |-- GeoLocation: string (nullable = true)
 |-- Premise: string (nullable = true)
 |-- VRIName: string (nullable = true)
 |-- Total_Incidents: integer (nullable = true)
 |-- Shape: string (nullable = true)



In [93]:
crime_df.head(5)

[Row(X=1420074.125, Y=594160.625, RowID=1, CrimeDateTime='2022/02/26 04:00:00+00', CrimeCode='4E', Location='200 W MONUMENT ST', Description='COMMON ASSAULT', Inside_Outside=None, Weapon='NA', Post=124, District='CENTRAL', Neighborhood='MOUNT VERNON', Latitude=39.29750061035156, Longitude=-76.61930084228516, GeoLocation='(39.2975,-76.6193)', Premise=None, VRIName=None, Total_Incidents=1, Shape=None),
 Row(X=1411374.25, Y=589791.375, RowID=2, CrimeDateTime='2022/02/26 01:26:30+00', CrimeCode='9S', Location='2100 FREDERICK AVE', Description='SHOOTING', Inside_Outside='Outside', Weapon='FIREARM', Post=835, District='SOUTHWEST', Neighborhood='CARROLLTON RIDGE', Latitude=39.28559875488281, Longitude=-76.65010070800781, GeoLocation='(39.2856,-76.6501)', Premise='COMMON BUSINESS', VRIName='Tri-District', Total_Incidents=1, Shape=None),
 Row(X=1411401.125, Y=582761.75, RowID=3, CrimeDateTime='2022/02/26 10:22:00+00', CrimeCode='4E', Location='2000 GRINNALDS AVE', Description='COMMON ASSAULT', 

In [94]:
crime_codes = crime_df.select("CrimeCode").distinct().show()

+---------+
|CrimeCode|
+---------+
|       3P|
|       3K|
|      3BJ|
|       1A|
|       3M|
|       5F|
|       4B|
|       3B|
|       7A|
|      3NF|
|      3EF|
|       3N|
|       5D|
|       6K|
|      3LO|
|      3AF|
|       7B|
|      3GO|
|     3AJF|
|      8GV|
+---------+
only showing top 20 rows



In [95]:
crime_codes_count = crime_df.select("CrimeCode").distinct().count()

In [96]:
crime_codes_count     # Distinct crime code count

85

In [97]:
crime_codes_num = crime_df.groupBy("CrimeCode").count().sort("count", ascending=0)


In [98]:
crime_codes_num.show()

+---------+-----+
|CrimeCode|count|
+---------+-----+
|       4E|91650|
|       6D|68427|
|       5A|43928|
|       7A|40274|
|       6J|27636|
|       6G|26858|
|       6E|24300|
|       6C|23227|
|       4C|22438|
|       5D|14971|
|      3AF|14739|
|       4B|14460|
|       4A|13226|
|       3B|10737|
|       4D| 7232|
|       5B| 6475|
|       9S| 5443|
|       6F| 5081|
|       5C| 4917|
|       6B| 4106|
+---------+-----+
only showing top 20 rows



In [99]:
crime_neighborhood = crime_df.groupBy("Neighborhood").count().sort("count", ascending=0).head(1)


In [100]:
crime_neighborhood   # displaying the max cases in Neigborhood

[Row(Neighborhood='DOWNTOWN', count=17799)]

In [110]:
from pyspark.sql.functions import *
Dates= crime_df.withColumn("CrimeDateTime",to_timestamp("CrimeDateTime", 'yyyy/MM/dd HH:mm:ss+00'))

In [130]:
crime_year = Dates.withColumn('CrimeDateTime_year',year(Dates.CrimeDateTime))
crime_month = crime_year.withColumn('CrimeDateTime_month',month(Dates.CrimeDateTime))

In [140]:
crime_dates=crime_month.select("CrimeDateTime_year", "CrimeDateTime_month").groupBy("CrimeDateTime_year", "CrimeDateTime_month").count().sort("count", ascending=0).show(5)

# 10th month of 2016 has the most crime rate

+------------------+-------------------+-----+
|CrimeDateTime_year|CrimeDateTime_month|count|
+------------------+-------------------+-----+
|              2016|                 10| 4810|
|              2017|                 10| 4797|
|              2015|                  7| 4684|
|              2017|                  8| 4683|
|              2017|                  7| 4641|
+------------------+-------------------+-----+
only showing top 5 rows



In [144]:
crime_dates=crime_month.select("CrimeDateTime_month").groupBy("CrimeDateTime_month").count().sort("count", ascending=0).show(1)
# Month which has the max crime rate

+-------------------+-----+
|CrimeDateTime_month|count|
+-------------------+-----+
|                  8|46327|
+-------------------+-----+
only showing top 1 row



In [141]:
weapons_distinct = crime_df.select("Weapon").where("Weapon != 'NA'").distinct().show()
# Weapons used

+-------+
| Weapon|
+-------+
|  HANDS|
|  KNIFE|
|  OTHER|
|   FIRE|
|FIREARM|
+-------+



In [142]:
weapons_most = crime_df.where("Weapon != 'NA'").groupBy("Weapon").count().sort("count", ascending=0).show(1)
# Most Used Wepon

+-------+-----+
| Weapon|count|
+-------+-----+
|FIREARM|46139|
+-------+-----+
only showing top 1 row

